In [1]:
!pip install -U langchain-text-splitters
!pip install -U langchain-community langchain-text-splitters langchain
!pip install pypdf
!pip install -U langchain-huggingface
!pip install -qU langchain-chroma
!pip install -qU langchain
!pip install -qU langchain-groq
!pip install sentence-transformers
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.7/496.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling langgraph-1.0.7:
      Successfully uninstalled langgraph-1.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.9
    Uninstalling langchain-1.2.9:
      Successfully uninstalled langchain-1.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, bu

In [2]:
# DAY 1
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [3]:
loader = PyPDFLoader("../content/sample_data/assurance.pdf")
pdf = loader.load()

print(f"Document chargé : {len(pdf)} pages trouvées.")

Document chargé : 8 pages trouvées.


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,      # Taille cible de chaque morceau
    chunk_overlap = 150,    # Chevauchement
    separators = ["\n\n", "\n", " ", ""] # L'ordre de priorité pour couper
)

chunks = text_splitter.split_documents(pdf)

In [5]:
print(f"Nombre total de chunks pour {len(pdf)} pages : {len(chunks)}")

lengths = [len(c.page_content) for c in chunks]
import statistics
print(f"Taille moyenne : {statistics.mean(lengths)} caractères")

# Voir à quelle page appartient le chunk n°10
print(f"Le chunk 10 vient de la page : {chunks[10].metadata['page']}")

Nombre total de chunks pour 8 pages : 31
Taille moyenne : 803.7096774193549 caractères
Le chunk 10 vient de la page : 2


In [6]:
# DAY 2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Création de la base de données physique
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=hf_embeddings,
    persist_directory="./chroma_db"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
question = "Quelles sont les garanties de mon contrat ?"
question_embedding = hf_embeddings.embed_documents([question])

# Demande les 'k' meilleurs résultats
# Chroma fait l'embedding de la question et le calcul de similarité
docs = vectordb.similarity_search(question, k=1)

print(f"Question : {question}")
print("---")
print(f"Réponse: {docs[0].page_content}")

Question : Quelles sont les garanties de mon contrat ?
---
Réponse: Ce contrat est SANS TACITE reconduction, et à renouveler chaque année avant le 1er septembre.
Il garantit l'assuré en tant que locataire (ou occupant à titre gratuit), de Appartement situé(e) à l’adresse suivante :
2 route de Narbonne – N° App: 167  - Résidence: Résidence Bordegrande – Cplt adresse :  – Code postal et Ville : 31320 AUZEVILLE-
TOLOSANE 
Il couvre les risques suivants : 
- INCENDIE (sans franchise)
- DEGATS DES EAUX (sans franchise)
- PROTECTION JURIDIQUE
- INDEMNITE REDOUBLEMENT (garantie acquise uniquement pour les étudiants)
- INDIVIDUELLE ACCIDENT (garantie acquise uniquement pour les jeunes diplômés)
Pour toutes les questions relatives à la gestion de votre contrat, à vos déclarations de sinistre ou à vos besoins d’assistance, vous pouvez
contacter le n° 03 20 33 09 33.
La compagnie atteste avoir pris connaissance du fait que les personnes assurées au contrat seront amenées au sein du logement assur

In [8]:
# DAY 3
from langchain_groq import ChatGroq
import os

# os.environ["GROQ_API_KEY"] = "..."
os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.5
)

In [9]:
from langchain_classic.chains import RetrievalQA

# Transformer la base Chroma en "chercheur" (retriever)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Créer la chaîne RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # "stuff" veut dire : "donne tout le texte au LLM"
    retriever=retriever
)

In [10]:
response = qa_chain.invoke("Quelles sont les garanties de mon contrat ?")
print(response["result"])

Selon le texte fourni, les garanties de votre contrat sont les suivantes :

1. Incendie (sans franchise)
2. Dégâts des eaux (sans franchise)
3. Protection juridique
4. Indemnité redouplement (garantie acquise uniquement pour les étudiants)
5. Individuelle accident (garantie acquise uniquement pour les jeunes diplômés)

Il est également mentionné que les dommages causés au matériel confié par l'employeur sont couverts au titre des garanties "dommages" acquises au contrat, mais il s'agit d'une garantie spécifique liée au télétravail et non d'une garantie générale du contrat.


In [11]:
# DAY 4

In [12]:
import gradio as gr
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain

# 1. Configurer la mémoire
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer'
)

/tmp/ipython-input-590994917.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [13]:
# 2. Créer la chaîne avec mémoire
qa_chat = ConversationalRetrievalChain.from_llm(
    llm=llm, # LLM Groq
    retriever=retriever, # retriever Chroma
    memory=memory
)

In [14]:
# 3. Fonction pour l'interface Gradio
def chat_interactif(message, history):
    response = qa_chat.invoke({"question": message})
    return response["answer"]

In [15]:
# 4. Lancer l'interface
demo = gr.ChatInterface(
    fn=chat_interactif,
    title="📚 Mon Assistant PDF Intelligent",
    description="Posez des questions sur votre document, je m'en souviendrai !",
    examples=["Fais-moi un résumé", "Quels sont les points clés ?"],
    theme="soft"
)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [16]:
demo.launch(share=True) # share=True crée un lien public de 72h !

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5383535cef36f54ee7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
# DAY 5

In [218]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
    input_key='question'
)


In [239]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

memory.clear()
# Définir les instructions (Prompt)
instruction_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Tu es un robot strictement limité au texte fourni.\n\n"
        "DOCUMENTS DE RÉFÉRENCE :\n"
        "{context}\n\n"
        "CONSIGNE:\n"
        "1. Ne réponds QUE si la réponse est écrite dans les 'DOCUMENTS DE RÉFÉRENCE' ci-dessus.\n"
        "2. Si tu ne trouves pas la réponse, réponds 'Je ne sais pas, cette information n'est pas dans le document."
        ". Sinon, donne la réponse DIRECTEMENT, sans expliquer tes consignes ni tes limites."
        "3. INTERDICTION d'utiliser tes propres connaissances.\n"
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

In [240]:
qa_chat = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": instruction_prompt},
    # forcer l'historique à rester une liste
    get_chat_history=lambda h: h
)

In [241]:
def chat_interactif(message, history):
    # Appeler la chaîne pour obtenir le résultat complet
    result = qa_chat.invoke({"question": message})

    # Extraire le texte de la réponse
    reponse_texte = result.get("answer", "Pas de réponse trouvée.")

    if "ne sais pas" in reponse_texte.lower():
        return reponse_texte

    # Extraire les documents sources
    sources = result.get("source_documents", [])

    # Vérifier la présence de sources pour extraire les pages
    if sources:
        # Récupérer les numéros de pages uniques et les trier
        pages = sorted(list(set([str(int(d.metadata.get("page", 0)) + 1) for d in sources])))

        # Formater et ajouter les pages à la fin du texte
        suffixe_source = f"\n\n(Source : Page {', '.join(pages)})"
        reponse_texte += suffixe_source

    return reponse_texte

In [242]:
#chat_interactif("fais moi résume", [])

In [243]:
demo = gr.ChatInterface(
    fn=chat_interactif,
    title="📚 Assistant PDF Pro",
    description="Analyse de documents avec citations de sources",
    examples=["Fais-moi un résumé", "Quels sont les points clés ?"],
    theme="soft"
)
demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3cdb91aacbf66d777.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
